# KOD DLA SIECI NEURONOWEJ 3 8 1

# Funkcje oraz klasy sieci

Czemu w zasadzie służy backrpopagation? 

Algorytm propagacji wstecznej błędu pomaga sieci neuronowej nauczyć się, jak dostosować swoje wagi, aby lepiej pasowały do danych treningowych. To jakbyśmy chcieli nauczyć kogoś gry na gitarze: jeśli osoba gra fałszywie, mówimy jej, gdzie popełniła błąd i jak to poprawić. W przypadku sieci neuronowej, algorytm ten mówi nam, jak zmienić każdą wagę, aby błąd sieci był coraz mniejszy. W ten sposób sieć staje się coraz lepsza w wykonywaniu swojego zadania, czy to rozpoznawanie obrazów, czy tłumaczenie języków.

In [2]:
import numpy as np 

def relu(x):
        return x * (x > 0)

def relu_derivative(x):
    return 1. * (x > 0)

def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)


#Klasa dla sieci 3-8-1
class NeuralNetwork:

    #Inicjalizacja sieci
    def __init__(self, x, y, seed, eta):
        np.random.seed(seed)
        self.input = x.T
        # Wagi z 3 wejsc do warstwy ukrytej z 8 neuronami
        self.weights1 = np.random.rand(self.input.shape[0],8).T
        #print(self.weights1)
        # Wagi z warstwy ukrytej do warstwy wyjsciowej
        self.weights2 = np.random.rand(8,1).T
        #print(self.weights2)
        # Wartosci wyjsciowe        
        self.y = y.T
        self.output = np.zeros(self.y.shape)
        #print(self.output)
        self.eta = eta

    #Feedforward z funkcjami aktywacji jako argumenty dla tworzenia kombinacji
    def feedforward(self, activation_function_1, activation_function_2):
        self.layer1 = activation_function_1(np.dot(self.weights1, self.input))
        self.output = activation_function_2(np.dot(self.weights2, self.layer1))

    #Propagacja wsteczna z funkcjami aktywacji jako argumenty dla tworzenia kombinacji
    def backprop(self, derivative_activation_function_1, derivative_activation_function_2):
        # Blad dla warstwy wyjsciowej
        delta_2 = (self.y - self.output) * derivative_activation_function_2(self.output)
        d_weights2 = self.eta * np.dot(delta_2, self.layer1.T)
        # Blad dla warstwy ukrytej
        delta_1 = derivative_activation_function_1(self.layer1) * np.dot(self.weights2.T, delta_2)
        d_weights1 = self.eta * np.dot(delta_1, self.input.T)
        # Aktualizacja wag
        self.weights1 += d_weights1
        self.weights2 += d_weights2
    
    # Algorytm propagacji wstecznej wykonujacy sie przez kreslona liczbe epok
    def train(self, f1, f2, epochs):
        activation_function_1 = relu
        derivative_activation_function_1 = relu_derivative
        activation_function_2 = relu
        derivative_activation_function_2 = relu_derivative

        if f1 == 'sigmoid':
            activation_function_1 = sigmoid
            derivative_activation_function_1 = sigmoid_derivative
        if f2 == 'sigmoid':
            activation_function_2 = sigmoid
            derivative_activation_function_2 = sigmoid_derivative
        for _ in range(epochs):
            self.feedforward(activation_function_1, activation_function_2)
            self.backprop(derivative_activation_function_1, derivative_activation_function_2)

    #Algorytm propagacji wstecznej wykonujacy sie do osiagniecia warunku konca
    def train_cond(self, f1, f2, cond):
        activation_function_1 = relu
        derivative_activation_function_1 = relu_derivative
        activation_function_2 = relu
        derivative_activation_function_2 = relu_derivative
        epochs = 0
        if f1 == 'sigmoid':
            activation_function_1 = sigmoid
            derivative_activation_function_1 = sigmoid_derivative
        if f2 == 'sigmoid':
            activation_function_2 = sigmoid
            derivative_activation_function_2 = sigmoid_derivative
        while True:
            self.feedforward(activation_function_1, activation_function_2)
            self.backprop(derivative_activation_function_1, derivative_activation_function_2)
            epochs += 1
            if cond(epochs, self.output):
                break
        return epochs
    
# Klasa z wykorzystaniem torch dla sieci 3-8-1 - celem pornwania z poprzednia
import torch
import torch.nn as nn
import torch.optim as optim

class NeuralNetworkTorch(nn.Module):
    def __init__(self, seed, eta):
        super(NeuralNetworkTorch, self).__init__()
        self.fc1 = nn.Linear(3, 8)
        self.fc2 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        np.random.seed(seed)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.SGD(self.parameters(), lr=eta)
    
    def forward(self, x, f1, f2):
        if f1 == 'sigmoid':
            x = self.sigmoid(self.fc1(x))
        else:
            x = self.relu(self.fc1(x))
        if f2 == 'sigmoid':
            x = self.sigmoid(self.fc2(x))
        else:
            x = self.relu(self.fc2(x))
        return x
    
    def train(self, X, y, f1, f2, epochs):
        for _ in range(epochs):
            self.optimizer.zero_grad()
            output = self.forward(X, f1, f2)
            loss = self.criterion(output, y)
            loss.backward()
            self.optimizer.step()
    
    def train_cond(self, X, y, f1, f2, cond):
        epochs = 0
        while True:
            self.optimizer.zero_grad()
            output = self.forward(X, f1, f2)
            loss = self.criterion(output, y)
            loss.backward()
            self.optimizer.step()
            epochs += 1
            if cond(epochs, output.detach().numpy().T):
                break
        return epochs


# Funkcje do testowania

In [4]:
# Test do znajdowania kombinacji zwracajacej najlepsze wyniki dla roznych eta - iteruje przez seedy
def accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs):
    print("Test dokladnosci ")
    for eta in etas:
        print(" ")
        print("Dla eta = " + str(eta))
        results = np.array([[1.,0.,0.,1.],[1.,0.,0.,1.], [1.,0.,0.,1.], [1.,0.,0.,1.]]) #Celowe zle wyniki
        best_seeds = np.array([0, 0, 0, 0])
        best_combinations = []

        for combination_id in range(len(combinations)):
            comb = combinations[combination_id]
            for seed in range(min_seed, max_seed):
                nn = NeuralNetwork(X, y, seed, eta)
                nn.train(comb[0], comb[1], epochs)
                if abs(nn.output - expected).sum() < abs(results[combination_id] - expected).sum():
                    results[combination_id] = nn.output
                    best_seeds[combination_id] = seed
            best_combinations.append((results[combination_id], combination_id))

        best_combinations.sort(key=lambda x: abs(x[0] - expected).sum())

        for i in range(len(best_combinations)):
            result = best_combinations[i][0]
            comb_id = best_combinations[i][1]
            comb = combinations[comb_id]
            print("Kombinacja: " + str(comb[0]) + " " + str(comb[1]) + " seed = " + str(best_seeds[comb_id]))
            print("Wynik: " + str(result) + " Blad: " + str(abs(result - expected).sum()))

# Analogiczny test z wykorzystaniem biblioteki torch
def accuracy_test_torch(X, y, min_seed, max_seed, etas, combinations, expected, epochs):
    print("Test dokladnosci ")
    for eta in etas:
        print(" ")
        print("Dla eta = " + str(eta))
        results = np.array([[1.,0.,0.,1.],[1.,0.,0.,1.], [1.,0.,0.,1.], [1.,0.,0.,1.]]) #Celowe zle wyniki
        best_seeds = np.array([0, 0, 0, 0])
        best_combinations = []

        for combination_id in range(len(combinations)):
            comb = combinations[combination_id]
            for seed in range(min_seed, max_seed):
                nn = NeuralNetworkTorch(seed, eta)
                nn.train(X.clone(), y.clone(), comb[0], comb[1], epochs)
                output = nn.forward(X.clone(), comb[0], comb[1]).detach().numpy().T
                #print(output)
                if abs(output - expected).sum() < abs(results[combination_id] - expected).sum():
                    results[combination_id] = output
                    best_seeds[combination_id] = seed
            best_combinations.append((results[combination_id], combination_id))

        best_combinations.sort(key=lambda x: abs(x[0] - expected).sum())

        for i in range(len(best_combinations)):
            result = best_combinations[i][0]
            comb_id = best_combinations[i][1]
            comb = combinations[comb_id]
            print("Kombinacja: " + str(comb[0]) + " " + str(comb[1]) + " seed = " + str(best_seeds[comb_id]))
            print("Wynik: " + str(result) + " Blad: " + str(abs(result - expected).sum()))


# Test do znajdowania kombinacji zwracajacej wzglednie poprawny wynik w najszybszym czasie
def time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs):
    print("Test czasu")
    for eta in etas:
        print(" ")
        print("Dla eta = " + str(eta))
        results = np.array([0, 0, 0, 0])
        best_seeds = np.array([0, 0, 0, 0])
        best_combinations = []
        for combination_id in range(len(combinations)):
            comb = combinations[combination_id]
            for seed in range(min_seed, max_seed):
                nn = NeuralNetwork(X, y, seed, eta)
                ep = nn.train_cond(comb[0], comb[1], lambda epochs, output: abs(output - expected).sum() < 0.1 or epochs > max_epochs)
                if(ep < results[combination_id] or results[combination_id] == 0):
                    results[combination_id] = ep
                    best_seeds[combination_id] = seed
            best_combinations.append((results[combination_id], combination_id))

        best_combinations.sort(key=lambda x: x[0])

        for i in range(len(best_combinations)):
            result = best_combinations[i][0]
            comb_id = best_combinations[i][1]
            comb = combinations[comb_id]
            print("Kombinacja: " + str(comb[0]) + " " + str(comb[1]) + " seed = " + str(best_seeds[comb_id]) + " epoki = " + str(result))


# Analogiczny test z wykorzystaniem biblioteki torch
def time_test_torch(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs):
    print("Test czasu")
    for eta in etas:
        print(" ")
        print("Dla eta = " + str(eta))
        results = np.array([0, 0, 0, 0])
        best_seeds = np.array([0, 0, 0, 0])
        best_combinations = []
        for combination_id in range(len(combinations)):
            comb = combinations[combination_id]
            for seed in range(min_seed, max_seed):
                nn = NeuralNetworkTorch(seed, eta)
                ep = nn.train_cond(X, y, comb[0], comb[1], lambda epochs, output: abs(output - expected).sum() < 0.1 or epochs > max_epochs)
                if(ep < results[combination_id] or results[combination_id] == 0):
                    results[combination_id] = ep
                    best_seeds[combination_id] = seed
            best_combinations.append((results[combination_id], combination_id))

        best_combinations.sort(key=lambda x: x[0])

        for i in range(len(best_combinations)):
            result = best_combinations[i][0]
            comb_id = best_combinations[i][1]
            comb = combinations[comb_id]
            print("Kombinacja: " + str(comb[0]) + " " + str(comb[1]) + " seed = " + str(best_seeds[comb_id]) + " epoki = " + str(result))

# Nauka XOR

In [19]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

y = np.array([[0],[1],[1],[0]])


Testy dokładności

In [21]:

max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,1.,1.,0.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 88
Wynik: [0.0759189  0.96499255 0.96505116 0.0237286 ] Blad: 0.1696037776554321
Kombinacja: relu sigmoid seed = 306
Wynik: [0.52984087 0.94943954 0.97111138 0.99821825] Blad: 1.607508204692079
Kombinacja: sigmoid relu seed = 50
Wynik: [0.34974604 0.53371252 0.50058982 0.60418861] Blad: 1.9196323065942416
Kombinacja: sigmoid sigmoid seed = 306
Wynik: [0.85553875 0.90216125 0.91272567 0.93565367] Blad: 1.976305503199579
 
Dla eta = 0.01
Kombinacja: relu relu seed = 385
Wynik: [1.52655666e-15 1.00000000e+00 1.00000000e+00 3.27515792e-15] Blad: 1.0019762797242038e-14
Kombinacja: relu sigmoid seed = 143
Wynik: [0.14440354 0.91191731 0.90975959 0.07283873] Blad: 0.3955653742611115
Kombinacja: sigmoid relu seed = 132
Wynik: [0.13001269 0.76173373 0.74935901 0.34649944] Blad: 0.9654193895471552
Kombinacja: sigmoid sigmoid seed = 50
Wynik: [0.46369846 0.51679619 0.50928873 0.53044823] Blad: 1.968061766485
 
Dla eta = 0.05
Kombina

Testy czasu

In [ ]:
max_epochs = 10000
etas = np.array([ 0.05, 0.1, 0.5])
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs) # okolo 15 min

Test czasu
 
Dla eta = 0.05
Kombinacja: relu relu seed = 48 epoki = 219
Kombinacja: sigmoid relu seed = 385 epoki = 1538
Kombinacja: relu sigmoid seed = 62 epoki = 5055
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.1
Kombinacja: sigmoid relu seed = 66 epoki = 1045
Kombinacja: relu sigmoid seed = 62 epoki = 2524
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.5
Kombinacja: relu sigmoid seed = 62 epoki = 504
Kombinacja: sigmoid sigmoid seed = 424 epoki = 3486
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001


Porownanie z torch - dokladnosc

In [ ]:
X = torch.tensor([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]], dtype = torch.float32)

y = torch.tensor([[0],[1],[1],[0]], dtype=torch.float32)

max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,1.,1.,0.]])
epochs = 5000
accuracy_test_torch(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 227m



Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 176
Wynik: [0.23495322 0.78556877 0.78164208 0.22766519] Blad: 0.8954075574874878
Kombinacja: relu sigmoid seed = 388
Wynik: [0.44344303 0.56811631 0.49765477 0.46961838] Blad: 1.8472903370857239
Kombinacja: sigmoid relu seed = 464
Wynik: [0.44132662 0.48096782 0.52469069 0.55136466] Blad: 1.9870327711105347
Kombinacja: sigmoid sigmoid seed = 161
Wynik: [0.47669002 0.49546635 0.48851928 0.50459677] Blad: 1.997301161289215
 
Dla eta = 0.01
Kombinacja: relu relu seed = 57
Wynik: [5.20050526e-06 9.99999642e-01 9.99995947e-01 0.00000000e+00] Blad: 9.611248970031738e-06
Kombinacja: relu sigmoid seed = 318
Wynik: [0.18124172 0.77244622 0.801871   0.251194  ] Blad: 0.8581185042858124
Kombinacja: sigmoid relu seed = 92
Wynik: [0.4601877  0.49278429 0.53574538 0.51717114] Blad: 1.948829174041748
Kombinacja: sigmoid sigmoid seed = 250
Wynik: [0.49493185 0.50371665 0.49637133 0.50129539] Blad: 1.996139258146286
 
Dla eta = 0.05
Komb

Porownanie z torch - czas

In [ ]:
max_epochs = 10000
etas = np.array([0.05, 0.1, 0.5])
time_test_torch(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)  # okolo 3m

Test czasu
 
Dla eta = 0.05
Kombinacja: relu relu seed = 496 epoki = 243
Kombinacja: sigmoid relu seed = 230 epoki = 3461
Kombinacja: relu sigmoid seed = 497 epoki = 9449
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.1
Kombinacja: relu relu seed = 257 epoki = 131
Kombinacja: sigmoid relu seed = 372 epoki = 1681
Kombinacja: relu sigmoid seed = 266 epoki = 4745
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.5
Kombinacja: relu relu seed = 180 epoki = 29
Kombinacja: relu sigmoid seed = 221 epoki = 918
Kombinacja: sigmoid sigmoid seed = 427 epoki = 6702
Kombinacja: sigmoid relu seed = 1 epoki = 10001


# Nauka AND 

In [22]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

y = np.array([[0],[0],[0],[1]])

Testy dokladnosci

In [23]:


max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,0.,0.,1.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 136
Wynik: [-0.          0.00535678  0.00493409  0.9930752 ] Blad: 0.017215675158616384
Kombinacja: sigmoid sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: relu sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: sigmoid relu seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
 
Dla eta = 0.01
Kombinacja: relu relu seed = 273
Wynik: [-0.00000000e+00  4.44089210e-16  2.44249065e-15  1.00000000e+00] Blad: 4.218847493575595e-15
Kombinacja: sigmoid relu seed = 405
Wynik: [-0.00000000e+00  7.03844642e-04  7.05640472e-04  9.98742447e-01] Blad: 0.0026670379792041454
Kombinacja: relu sigmoid seed = 35
Wynik: [0.00777242 0.06669031 0.06585321 0.921848  ] Blad: 0.21846794839377434
Kombinacja: sigmoid sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
 
Dla eta = 0.05
Kombinacja: relu relu seed = 281
Wynik: [-0.00000000e+00  1.52655666e-15  1.94289029e-16  1.00000000e+00] Blad: 2.831068712794149e-15
Kombinacja: sigmoid relu see

Testy czasu

In [24]:
max_epochs = 10000
etas = np.array([0.05, 0.1, 0.5])
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.05
Kombinacja: relu relu seed = 281 epoki = 153
Kombinacja: sigmoid relu seed = 136 epoki = 580
Kombinacja: relu sigmoid seed = 424 epoki = 2782
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.1
Kombinacja: relu sigmoid seed = 424 epoki = 1391
Kombinacja: sigmoid sigmoid seed = 226 epoki = 6589
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001
 
Dla eta = 0.5
Kombinacja: relu sigmoid seed = 424 epoki = 278
Kombinacja: sigmoid sigmoid seed = 226 epoki = 1316
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001


Porownanie z torch - dokladnosc

In [25]:
X = torch.tensor([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]], dtype = torch.float32)

y = torch.tensor([[0],[0],[0],[1]], dtype=torch.float32)

max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,0.,0.,1.]])
epochs = 5000
accuracy_test_torch(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 35
Wynik: [0.         0.05695994 0.07864721 0.88577777] Blad: 0.24982938170433044
Kombinacja: sigmoid sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: relu sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: sigmoid relu seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
 
Dla eta = 0.01
Kombinacja: relu relu seed = 21
Wynik: [0.00000000e+00 1.16229057e-06 4.05311584e-06 9.99996245e-01] Blad: 8.970499038696289e-06
Kombinacja: sigmoid relu seed = 45
Wynik: [0.         0.02347118 0.02390063 0.95387822] Blad: 0.09349358081817627
Kombinacja: relu sigmoid seed = 11
Wynik: [0.00473271 0.11473712 0.11286163 0.84909624] Blad: 0.3832352291792631
Kombinacja: sigmoid sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
 
Dla eta = 0.05
Kombinacja: relu relu seed = 6
Wynik: [0.00000000e+00 1.78813934e-07 1.78813934e-07 9.99999642e-01] Blad: 7.152557373046875e-07
Kombinacja: sigmoid relu seed = 12
Wynik: [0.00000000e+00 2.38418579e

Porownanie z torch - czas

In [26]:
max_epochs = 10000
etas = np.array([0.05, 0.1, 0.5])
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.05
Kombinacja: sigmoid relu seed = 1 epoki = 624
Kombinacja: relu sigmoid seed = 35 epoki = 2806
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.1
Kombinacja: relu sigmoid seed = 35 epoki = 1404
Kombinacja: sigmoid sigmoid seed = 12 epoki = 6862
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001
 
Dla eta = 0.5
Kombinacja: relu sigmoid seed = 35 epoki = 281
Kombinacja: sigmoid sigmoid seed = 12 epoki = 1373
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001


# Nauka OR

In [27]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

y = np.array([[0],[1],[1],[1]])

Testy dokladnosci

In [28]:


max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,1.,1.,1.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 88
Wynik: [0.07243831 0.95456155 0.95923225 1.0350522 ] Blad: 0.19369671765082153
Kombinacja: relu sigmoid seed = 306
Wynik: [0.53104478 0.9521709  0.97273273 0.99840853] Blad: 0.6077326190847117
Kombinacja: sigmoid relu seed = 50
Wynik: [0.47387663 0.82219595 0.78214514 0.99375187] Blad: 0.8757836671666757
Kombinacja: sigmoid sigmoid seed = 325
Wynik: [0.97458505 0.98594856 0.98874804 0.99234987] Blad: 1.007538579433461
 
Dla eta = 0.01
Kombinacja: relu relu seed = 385
Wynik: [3.48831797e-11 1.00000000e+00 1.00000000e+00 1.00000000e+00] Blad: 1.2384579473057045e-10
Kombinacja: relu sigmoid seed = 424
Wynik: [0.08671704 0.95806583 0.95307849 0.99979544] Blad: 0.1757772791624079
Kombinacja: sigmoid relu seed = 132
Wynik: [0.03689689 0.95341626 0.92704498 1.08142968] Blad: 0.2378653224485634
Kombinacja: sigmoid sigmoid seed = 483
Wynik: [0.50855198 0.7618689  0.76316914 0.87381901] Blad: 1.1096949283042354
 
Dla eta = 0.05


Testy czasu

In [29]:
max_epochs = 10000
etas = np.array([0.05, 0.1, 0.5])
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.05
Kombinacja: relu relu seed = 136 epoki = 134
Kombinacja: sigmoid relu seed = 132 epoki = 1665
Kombinacja: relu sigmoid seed = 424 epoki = 2088
Kombinacja: sigmoid sigmoid seed = 405 epoki = 9960
 
Dla eta = 0.1
Kombinacja: relu relu seed = 281 epoki = 234
Kombinacja: sigmoid relu seed = 64 epoki = 945
Kombinacja: relu sigmoid seed = 424 epoki = 1044
Kombinacja: sigmoid sigmoid seed = 405 epoki = 4980
 
Dla eta = 0.5
Kombinacja: relu sigmoid seed = 424 epoki = 209
Kombinacja: sigmoid sigmoid seed = 405 epoki = 997
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001


Porownanie z torch - dokladnosc

In [30]:
X = torch.tensor([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]], dtype = torch.float32)

y = torch.tensor([[0],[1],[1],[1]], dtype=torch.float32)

max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,1.,1.,1.]])
epochs = 5000
accuracy_test_torch(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 135
Wynik: [0.13946635 0.92179537 0.9045276  1.011814  ] Blad: 0.32495737075805664
Kombinacja: sigmoid relu seed = 159
Wynik: [0.5081166  0.78336132 0.73257476 0.98798817] Blad: 1.0041923522949219
Kombinacja: relu sigmoid seed = 260
Wynik: [0.53535891 0.72923523 0.7310887  0.85946172] Blad: 1.215573251247406
Kombinacja: sigmoid sigmoid seed = 430
Wynik: [0.75458449 0.75413519 0.75828654 0.75707108] Blad: 1.4850916862487793
 
Dla eta = 0.01
Kombinacja: relu relu seed = 92
Wynik: [8.94069672e-06 9.99995112e-01 9.99995112e-01 9.99995172e-01] Blad: 2.3543834686279297e-05
Kombinacja: relu sigmoid seed = 396
Wynik: [0.12687841 0.91636962 0.91644043 0.99665475] Blad: 0.2974136173725128
Kombinacja: sigmoid relu seed = 437
Wynik: [0.14996728 0.83526951 0.83854944 1.18238783] Blad: 0.6585361659526825
Kombinacja: sigmoid sigmoid seed = 127
Wynik: [0.60077924 0.73048776 0.72319341 0.81268907] Blad: 1.3344089984893799
 
Dla eta = 0.05

Porownanie z torch - czas 

In [6]:
X = torch.tensor([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]], dtype = torch.float32)

y = torch.tensor([[0],[1],[1],[1]], dtype=torch.float32)

max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,1.,1.,1.]])

max_epochs = 10000
etas = np.array([0.05, 0.1, 0.5])
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.05
Kombinacja: relu relu seed = 136 epoki = 134
Kombinacja: sigmoid relu seed = 132 epoki = 1665
Kombinacja: relu sigmoid seed = 424 epoki = 2088
Kombinacja: sigmoid sigmoid seed = 405 epoki = 9960
 
Dla eta = 0.1
Kombinacja: relu relu seed = 281 epoki = 234
Kombinacja: sigmoid relu seed = 64 epoki = 945
Kombinacja: relu sigmoid seed = 424 epoki = 1044
Kombinacja: sigmoid sigmoid seed = 405 epoki = 4980
 
Dla eta = 0.5
Kombinacja: relu sigmoid seed = 424 epoki = 209
Kombinacja: sigmoid sigmoid seed = 405 epoki = 997
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001


# Brak ostatniej kolumny

# XOR

In [7]:
X = np.array([
    [0,0,0],
    [0,1,0],
    [1,0,0],
    [1,1,0]
])

y = np.array([[0],[1],[1],[0]])
max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,1.,1.,0.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 244
Wynik: [0.         0.99571655 0.99614805 0.00443425] Blad: 0.012569656593053424
Kombinacja: relu sigmoid seed = 325
Wynik: [0.5        0.96873033 0.99052083 0.99969119] Blad: 1.5404400328021848
Kombinacja: sigmoid relu seed = 50
Wynik: [0.3157158  0.5257845  0.48301841 0.63318557] Blad: 1.9400984622215915
Kombinacja: sigmoid sigmoid seed = 306
Wynik: [0.83189188 0.88369873 0.896046   0.92284262] Blad: 1.974989761918351
 
Dla eta = 0.01
Kombinacja: relu relu seed = 111
Wynik: [0.00000000e+00 1.00000000e+00 1.00000000e+00 3.12250226e-16] Blad: 1.8665624601510444e-15
Kombinacja: relu sigmoid seed = 143
Wynik: [0.5        0.91800843 0.91950719 0.0992161 ] Blad: 0.7617004838992686
Kombinacja: sigmoid relu seed = 132
Wynik: [0.14184905 0.77531917 0.76518854 0.31159891] Blad: 0.9129402454675506
Kombinacja: sigmoid sigmoid seed = 50
Wynik: [0.46499218 0.51564006 0.50663117 0.5300663 ] Blad: 1.9727872466617475
 
Dla eta = 0.05

In [8]:
max_epochs = 10000
etas = np.array([ 0.05, 0.1, 0.5])
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs) # okolo 15 min

Test czasu
 
Dla eta = 0.05
Kombinacja: relu relu seed = 244 epoki = 66
Kombinacja: sigmoid relu seed = 385 epoki = 1962
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: relu sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.1
Kombinacja: relu relu seed = 143 epoki = 42
Kombinacja: sigmoid relu seed = 385 epoki = 996
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: relu sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.5
Kombinacja: sigmoid sigmoid seed = 424 epoki = 5985
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: relu sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001


# AND

In [9]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

y = np.array([[0],[0],[0],[1]])



max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,0.,0.,1.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 136
Wynik: [-0.          0.00535678  0.00493409  0.9930752 ] Blad: 0.017215675158616384
Kombinacja: sigmoid sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: relu sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: sigmoid relu seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
 
Dla eta = 0.01
Kombinacja: relu relu seed = 273
Wynik: [-0.00000000e+00  4.44089210e-16  2.44249065e-15  1.00000000e+00] Blad: 4.218847493575595e-15
Kombinacja: sigmoid relu seed = 405
Wynik: [-0.00000000e+00  7.03844642e-04  7.05640472e-04  9.98742447e-01] Blad: 0.0026670379792041454
Kombinacja: relu sigmoid seed = 35
Wynik: [0.00777242 0.06669031 0.06585321 0.921848  ] Blad: 0.21846794839377434
Kombinacja: sigmoid sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
 
Dla eta = 0.05
Kombinacja: relu relu seed = 281
Wynik: [-0.00000000e+00  1.52655666e-15  1.94289029e-16  1.00000000e+00] Blad: 2.831068712794149e-15
Kombinacja: sigmoid relu see

In [10]:
max_epochs = 10000
etas = np.array([0.05, 0.1, 0.5])
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.05
Kombinacja: relu relu seed = 281 epoki = 153
Kombinacja: sigmoid relu seed = 136 epoki = 580
Kombinacja: relu sigmoid seed = 424 epoki = 2782
Kombinacja: sigmoid sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.1
Kombinacja: relu sigmoid seed = 424 epoki = 1391
Kombinacja: sigmoid sigmoid seed = 226 epoki = 6589
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001
 
Dla eta = 0.5
Kombinacja: relu sigmoid seed = 424 epoki = 278
Kombinacja: sigmoid sigmoid seed = 226 epoki = 1316
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001


# OR

In [11]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

y = np.array([[0],[1],[1],[1]])


max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu"], ["sigmoid", "sigmoid"], ["relu", "sigmoid"], ["sigmoid", "relu"]])
expected = np.array([[0.,1.,1.,1.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu seed = 88
Wynik: [0.07243831 0.95456155 0.95923225 1.0350522 ] Blad: 0.19369671765082153
Kombinacja: relu sigmoid seed = 306
Wynik: [0.53104478 0.9521709  0.97273273 0.99840853] Blad: 0.6077326190847117
Kombinacja: sigmoid relu seed = 50
Wynik: [0.47387663 0.82219595 0.78214514 0.99375187] Blad: 0.8757836671666757
Kombinacja: sigmoid sigmoid seed = 325
Wynik: [0.97458505 0.98594856 0.98874804 0.99234987] Blad: 1.007538579433461
 
Dla eta = 0.01
Kombinacja: relu relu seed = 385
Wynik: [3.48831797e-11 1.00000000e+00 1.00000000e+00 1.00000000e+00] Blad: 1.2384579473057045e-10
Kombinacja: relu sigmoid seed = 424
Wynik: [0.08671704 0.95806583 0.95307849 0.99979544] Blad: 0.1757772791624079
Kombinacja: sigmoid relu seed = 132
Wynik: [0.03689689 0.95341626 0.92704498 1.08142968] Blad: 0.2378653224485634
Kombinacja: sigmoid sigmoid seed = 483
Wynik: [0.50855198 0.7618689  0.76316914 0.87381901] Blad: 1.1096949283042354
 
Dla eta = 0.05


In [12]:
max_epochs = 10000
etas = np.array([0.05, 0.1, 0.5])
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.05
Kombinacja: relu relu seed = 136 epoki = 134
Kombinacja: sigmoid relu seed = 132 epoki = 1665
Kombinacja: relu sigmoid seed = 424 epoki = 2088
Kombinacja: sigmoid sigmoid seed = 405 epoki = 9960
 
Dla eta = 0.1
Kombinacja: relu relu seed = 281 epoki = 234
Kombinacja: sigmoid relu seed = 64 epoki = 945
Kombinacja: relu sigmoid seed = 424 epoki = 1044
Kombinacja: sigmoid sigmoid seed = 405 epoki = 4980
 
Dla eta = 0.5
Kombinacja: relu sigmoid seed = 424 epoki = 209
Kombinacja: sigmoid sigmoid seed = 405 epoki = 997
Kombinacja: relu relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu seed = 1 epoki = 10001
